In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, dash_table
import pandas as pd
import base64
import plotly.express as px
import dash_leaflet as dl
from CRUD import CRUD_Operations

# Initialize CRUD operations.
crud_operations = CRUD_Operations("aacuser", "xyz123")

# Read in mongo db data and convert to pandas data frame.
df = pd.DataFrame(list(crud_operations.read({})))

# Drop Id column from pandas datframe.
df.drop(columns=['_id'], inplace=True)  

# Init the Dash app.
app = JupyterDash(__name__)

# Load and encode Grazioso company logo.
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Define layout of the app.
app.layout = html.Div([
    # Anchor log for URL, customize img attributes
    html.A([
        html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image), alt='customer image',
                 style={
                     'height': '7%',
                     'width': '7%',
                     'float': 'left',
                     'position': 'relative',
                     'padding-top': 0,
                     'padding-right': 0
                 }),
        # SNHU home page
    ], href='https://www.snhu.edu'),
    
    #Dubplicate image and attributes
    html.A([
        html.Img(id='customer-image-right', src='data:image/png;base64,{}'.format(encoded_image), alt='customer image',
                 style={
                     'height': '7%',
                     'width': '7%',
                     'float': 'right',
                     'position': 'relative',
                     'padding-top': 0,
                     'padding-right': 0
                 }),
    ], href='https://www.snhu.edu'),

    # Main Header Title
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    # Company Mission
    html.P("Grazioso Salvare is a pioneering Texas based company specializing in training dogs for search-and-rescue missions. "
           "Partnering with a network of regional animal shelters, Grazioso Salvare identifies and selects young dogs with specific "
           "profiles suited for various rescue scenarios, such as water or wilderness rescues.",
           style={'text-align': 'center', 'font-size': '16px', 'margin-top': '10px'}),
    # Unique Identifier for Contributo Name
    html.Center(html.B(html.H5(style={'margin-bottom': '5px'}, children="Global Rain Contributor"))),
    html.Center(html.B(html.H5(style={'margin-top': '0px', 'margin-bottom': '0px'}, children='Daniel Dickinson'))),
    # Seperation from Header to database
    html.Hr(),
    
    # Leaflet Radio button for searchable data
    html.Div(style={'marginBottom': '10px'}, children=[
        dcc.RadioItems(
            id='filter-type-radio',
            options=[
                # Rescue animal types
                {'label': 'Water', 'value': 'wtr'},
                {'label': 'Mountain/Wilderness', 'value': 'mnt'},
                {'label': 'Disaster/Tracker', 'value': 'dis'},
                {'label': 'Reset', 'value': 'res'},
            ],
            labelStyle={'display': 'inline-block', 'margin-right': '10px'},
            style={'text-align': 'center'}
        ),
    ]),

    # Configure Data table 
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10, # Pagnation.
    ),

    # Map container with styling.
    html.Div(style={'flex': '1', 'width': '100%', 'height': '50vh'}, children=[
        dl.Map(style={'width': '100%', 'height': '100%'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")
        ], id='map-id')
    ]),

    # Pie Chart container with styling.
    html.H3("Thanks to YOU! Adoption rate has increased.", style={'text-align': 'center', 'margin-top': '20px'}),
    html.Div(style={'flex': '1', 'width': '100%', 'height': '40vh'}, children=[
        dcc.Graph(id='pie-chart')
    ]),
])

# Radio Button Callback to filter data for data table.
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type-radio', 'value')]
)
def filter_table(filter_value):
    # Define query based on Rescue criteria
    if filter_value == 'wtr':
        query = {
            '$and': [
                {'sex_upon_outcome': 'Intact Female'},
                {'$or': [
                    {'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesapeake Bay Retriever Mix'},
                    {'breed': 'Newfoundland'},
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        }
        #Mountain and Wilderness Rescue criteria
    elif filter_value == 'mnt':
        query = {
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'$or': [
                    {'breed': 'German Shepherd'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'},
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        }
        # Disaster / Tracker Rescue criteria.
    elif filter_value == 'dis':
        query = {
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'},
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 20}},
                    {'age_upon_outcome_in_weeks': {'$lte': 300}}
                ]}
            ]
        }
    else:
        query = {}  # Return empty, if not option exist.

    # Get filtered data from Mongo and convert to pandas data frame.
    filtered = pd.DataFrame(list(crud_operations.read(query)))
    
    #TODO: Duplicating drop Id column, but causes CallBack error...
    filtered.drop(columns=['_id'], inplace=True)  # Drop id column

    # Return filtered data as a dict.
    return filtered.to_dict('records')

# Callback to update map from filtered data
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data'),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
    if selected_rows:
        # Convert viewData to DataFrame and get the selected row.
        dff = pd.DataFrame.from_dict(viewData)
        selected_row_index = selected_rows[0]
        selected_animal = dff.iloc[selected_row_index]
        
        # Create a marker for the selected animal.
        marker = dl.Marker(
            position=[selected_animal['location_lat'], selected_animal['location_long']],
            children=[
                # Display name of animal 
                dl.Tooltip(selected_animal['name']),
                dl.Popup([
                    html.H4("Animal Name"),
                    html.P(selected_animal['name']),
                ])
            ]
        )

        # Return map with selected animal
        return dl.Map(
            style={'width': '100%', 'height': '100%'},
            center=[selected_animal['location_lat'], selected_animal['location_long']], zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                marker
            ]
        )
    else:
        # Return map to "center" when no animal is selected
        return dl.Map(
            style={'width': '100%', 'height': '100%'},
            center=[30.75, -97.48], zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id")
            ]
        )

# Callback to update pie chart based on filtered data
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', 'derived_viewport_data')]
)

def update_pie_chart(viewData):
    # If no data is avail, return empty chart
    if viewData is None or len(viewData) == 0:
        return px.pie()  
    
    # # Convert viewData to DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    if 'outcome_type' not in dff.columns or dff['outcome_type'].empty:
        return px.pie()
    
    # Create the pic chart with collections outcome_type data
    names = dff['outcome_type'].value_counts().index.tolist()
    values = dff['outcome_type'].value_counts().tolist()
    fig = px.pie(data_frame=dff, values=values, names=names, color_discrete_sequence=px.colors.sequential.RdBu)
    return fig

# Run the Dash app
app.run_server()

Dash app running on http://127.0.0.1:20056/
